In [13]:
# Import libraries
from datetime import datetime, timedelta, time

import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [14]:
# For the global variables
FILE = 'folder/working_hours.txt'

TOTAL_MINUTES_HOUR = 60
DAILY_HOURS_TO_COVER = 8
NUM_WORKING_DAYS = 5

TOTAL_HOURS_TO_COVER = NUM_WORKING_DAYS * DAILY_HOURS_TO_COVER
MAX_MINUTES_RECOMMENDED = TOTAL_MINUTES_HOUR * DAILY_HOURS_TO_COVER
MAX_MINUTES_WEEKLY = TOTAL_MINUTES_HOUR * TOTAL_HOURS_TO_COVER
MAX_MINUTES_OVERTIME = TOTAL_MINUTES_HOUR * (DAILY_HOURS_TO_COVER + 1)

In [15]:
def get_current_time():
    return str(datetime.now().hour) + ':' + str(datetime.now().minute)

# Convert minutes to hours and minutes (in string)
def get_hours_minutes(total_minutes):

    hours = int(total_minutes/60)
    minutes = int(total_minutes % 60)

    return str(hours) + 'h ' + str(minutes) + 'm'

# Find the minutes between two times
# (Ex. 8:00-10:23)
def get_duration(times):
    
    minutes_covered = 0

    for hours in times:
        hours = hours.rstrip('\n')
        
        (start_time, end_time) = hours.split('-')
        FMT = '%H:%M'
        
        # Each time gap has two outcomes: 
            # 1. 1400-1800 (240 minutes) 
            # 2. 0800- (Current time)
        end_time = end_time if end_time != '' else get_current_time()

        # Get the time difference in minutes
        diff = datetime.strptime(end_time, FMT) - datetime.strptime(start_time, FMT)
        minutes_covered += diff.seconds/60

    return int(minutes_covered)

def get_day_number():
    return datetime.today().weekday()

# Based on the time covered, check:
# - how much time is remaining, or
# - how much overtime is happening
def find_today_hours_remaining(remaining_today):
    
    # Worst case: 8 hours not fulfilled yet
    covered_today = 0
    difference = remaining_today
    buffer_name = 'remaining'
    buffer_color = 'rgba(237, 49, 49, 0.6)'
    
    if remaining_today > 0:
    
        covered_today = MAX_MINUTES_RECOMMENDED - remaining_today

        finishing_time_today = datetime.now() + timedelta(minutes = remaining_today)
        print('FINISHING TIME TODAY (8 HOURS): ', finishing_time_today.time().strftime("%H:%M:%S"))
    else:

        covered_today = MAX_MINUTES_RECOMMENDED
        difference = abs(remaining_today)
        buffer_name = 'overtime'
        buffer_color = 'rgba(49, 237, 86, 0.6)'
    
    return (covered_today, difference, buffer_name, buffer_color)
    

# Find the average time needed to cover on a daily basis 
# to fulfill 40hours
def find_average_time_remaining(total_covered, days_array):
    
    day_number = get_day_number()
    
    # To avoid dynamic average:
        # 1. Get the total hours remaining
        # 2. Deduct today's hours
    total_time_exclude_today = MAX_MINUTES_WEEKLY - (total_covered - days_array[day_number]['minutes_covered'])
    days_remaining = NUM_WORKING_DAYS - day_number

    # Find the average and increment it with the remainder as well
    # REASON: Better to cover more minutes than not to
    avg_mins, remainder = divmod(total_time_exclude_today, (days_remaining))
    total_avg = avg_mins + remainder
    
    avg_hour, avg_mins = divmod(total_avg, TOTAL_MINUTES_HOUR)
    
    time_to_cover = str(avg_hour) + 'h ' + str(avg_mins) + 'm'

    return total_avg, time_to_cover

def get_weekly_hours_stats():
    
    total_covered = 0
    remaining_today = 0
    remaining_weekly = 0
    days_array = []
    
    day_number = get_day_number()

    with open(FILE) as file:

        # One line = Day, Time gap #1, Time gap #2, ..., Time gap n
        for index, line in enumerate(file):
            
            line_array = line.replace('\x00','').rstrip().split(',')
            
            day = line_array[0]
            day_coverage = line_array[1:]

            minutes_covered = get_duration(day_coverage)
            
            days_array.append({
                'day': day,
                'minutes_covered': minutes_covered,
                'hours_covered': get_hours_minutes(minutes_covered),
                'coverage': day_coverage
            })
            
            if index == day_number:
                
                remaining_today = MAX_MINUTES_RECOMMENDED - minutes_covered

            total_covered += minutes_covered

    # Objective is to know if 40 hours has been covered or not
    # Hence, overtime does not matter overall
    if total_covered < MAX_MINUTES_WEEKLY:
        remaining_weekly = MAX_MINUTES_WEEKLY - total_covered
    else:
        total_covered = MAX_MINUTES_WEEKLY
        
    total_covered = total_covered if total_covered <= MAX_MINUTES_WEEKLY else MAX_MINUTES_WEEKLY

    return (total_covered, days_array, remaining_today, remaining_weekly)

In [29]:
'''
There are three scenarios:
- 08:00-11:30
- 08:00-17:00
- 14:00-17:00

Analysis:

- Check if the second time is after noon
-- If it is NOT, then we don't care about the first time
-- If it is, check if the first time is after noon

Let's take 08:00-11:30 as an example:
- 11:30 >= 12:00 -- FALSE
- Do between 08:00-11:30

Let's take 08:00-17:00 as an example:
- 17:00 >= 12:00 -- TRUE
- 08:00 >= 12:00 - FALSE
- Do between 08:00-12:00
- Do between 12:00-17:00

'''

def analyze_noon_times(start_time, end_time):
    
    # Check if end time is after noon
    end_time_hour = int(end_time.split(':')[0])
    start_time_hour = int(start_time.split(':')[0])
    
    mid_day_time = time(12,00)
    
    best_case_answer = get_duration([ start_time + '-' + end_time])
    
    before_noon_minutes, after_noon_minutes = 0,0
    
    print(start_time_hour)
    print(end_time_hour)
    
    if time(end_time_hour,00) >= mid_day_time:
        
        if time(start_time_hour,00) >= mid_day_time:
            after_noon_minutes = best_case_answer
        else:
            before_noon_minutes += get_duration([ start_time + '-12:00']) 
            after_noon_minutes += get_duration([ '12:00-' + end_time])
            
    else:
        before_noon_minutes = best_case_answer
    
    return before_noon_minutes, after_noon_minutes

before_noon_minutes, after_noon_minutes = analyze_noon_times('13:00','15:30')
print(before_noon_minutes)
print(after_noon_minutes)

13
15
0
150


In [5]:
day_number = get_day_number()

(total_covered, days_array, remaining_today, remaining_weekly) = get_weekly_hours_stats()

(avg_minutes, hours_minutes_string) = find_average_time_remaining(total_covered, days_array)

(covered_today, difference, buffer_name, buffer_color) = find_today_hours_remaining(remaining_today)

days_df = pd.DataFrame(days_array)

FINISHING TIME TODAY (8 HOURS):  17:17:39


In [18]:
fig = go.Figure(go.Bar(
            x=days_df['minutes_covered'],
            y=days_df['day'],
            orientation='h'))

# Recommended: 480 minutes
fig.add_shape(
        dict(
            type="line",
            x0=MAX_MINUTES_RECOMMENDED,
            x1=MAX_MINUTES_RECOMMENDED,
            y0=-0.5,
            y1=4.5,
            line=dict(
                color="Red",
                width=2,
                dash="dot"
            )
))

# Average time to cover
fig.add_shape(
        dict(
            type="line",
            x0=avg_minutes,
            x1=avg_minutes,
            y0=-0.5,
            y1=4.5,
            line=dict(
                color="#FECB52",
                width=2,
                dash="dot"
            ),
))

fig.update_layout(
    title_text='Weekly hours calculation (Individual Days)',
    xaxis_title='Minutes covered',
    yaxis_title='Day',
    yaxis=dict(autorange="reversed"),
    width=1000,
    height=600,
    xaxis = dict(
      range=[0,600],
        tick0 = 0,
        dtick = 60
    )
)

fig.show()

In [7]:
fig = go.Figure()

fig.add_trace(go.Bar(
    y=['Today'],
    x=[covered_today],
    name='covered',
    orientation='h',
    marker=dict(
        color='rgba(246, 78, 139, 0.6)',
    )
))

fig.add_trace(go.Bar(
    y=['Today'],
    x=[difference],
    name=buffer_name,
    orientation='h',
    marker=dict(
        color=buffer_color,
    )
))

fig.update_layout(
    barmode='stack',
    autosize=False,
    width=800,
    height=300
)

fig.update_yaxes(automargin=True)

fig.show()

In [8]:
# Convert to dataframe
total_calculation_df = pd.DataFrame([
    {
        'category': 'covered', 
        'amount': total_covered, 
        'amount_hrs': get_hours_minutes(total_covered)
    },
    {
        'category': 'remaining', 
        'amount': remaining_weekly,
        'amount_hrs': get_hours_minutes(remaining_weekly)
    }
])

print(total_calculation_df)

    category  amount amount_hrs
0    covered     216     3h 36m
1  remaining    2184    36h 24m


In [9]:
# Pie chart
fig = go.Figure(go.Pie(
    name="",
    values = total_calculation_df['amount'],
    labels = total_calculation_df['category'],
    customdata = total_calculation_df['amount_hrs'],
    hovertemplate = "Category: %{label} <br>Total(minutes): %{value} </br>Total(hours): %{customdata}"

))

fig.update_layout(title_text='Overall weekly hours calculation: Remaining vs Covered')

fig.show()